# Results of all benchmark

| GPU                    | preprocess ms | inference ms | loss ms | postprocess ms | mask map50-95 | mask map50 | mask map75 |
|------------------------|---------------|--------------|---------|----------------|---------------|------------|------------|
| PyTorch fuse           | 0.1           | 2.8          | 0       | 0.3            | 0.37          | 0.577      | 0.395      |
| ONNX                   | 0.4           | 89.3         | 0       | 1.1            | 0.37          | 0.577      | 0.395      |
| TensorRT               | 0.2           | 3.2          | 0       | 0.5            | 0.37          | 0.577      | 0.395      |
| TensorRT simplify      | 0.2           | 3.2          | 0       | 0.5            | 0.37          | 0.577      | 0.395      |
| TensorRT simplify half | 0.1           | 1.5          | 0       | 0.5            | 0.371         | 0.579      | 0.397      |
| CPU                    |               |              |         |                |               |            |            |
| PyTorch fuse           | 0.5           | 78.2         | 0       | 0.6            | 0.371         | 0.58       | 0.396      |
| ONNX                   | 0.4           | 90.9         | 0       | 1.1            | 0.37          | 0.577      | 0.395      |
| ONNX Simplify          | 0.4           | 85.8         | 0       | 1.1            | 0.37          | 0.577      | 0.395      |
| ONNX Simplify half     | 0.5           | 87.7         | 0       | 1.3            | 0.37          | 0.577      | 0.395      |
| OpenVino CPU           | 0.4           | 60.1         | 0       | 1.3            | 0.37          | 0.577      | 0.395      |
| OpenVino CPU half      | 0.5           | 60.2         | 0       | 1.2            | 0.37          | 0.577      | 0.395      |


# PyTorch Fuse

In [1]:
import ultralytics
import shutil

ultralytics.checks()

Ultralytics YOLOv8.0.200 🚀 Python-3.10.13 torch-2.1.0+cu121 CUDA:0 (NVIDIA A10, 24074MiB)
Setup complete ✅ (12 CPUs, 31.1 GB RAM, 297.6/913.8 GB disk)


In [2]:
from ultralytics import YOLO
import os

# Load a model
# build a new model from scratch
model = YOLO("yolov8s-seg.yaml")
# load a pretrained model (recommended for training)
model = YOLO("yolov8s-seg.pt")


                   from  n    params  module                                       arguments                     
  0                  -1  1       928  ultralytics.nn.modules.conv.Conv             [3, 32, 3, 2]                 
  1                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  2                  -1  1     29056  ultralytics.nn.modules.block.C2f             [64, 64, 1, True]             
  3                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  4                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  5                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128, 256, 3, 2]              
  6                  -1  2    788480  ultralytics.nn.modules.block.C2f             [256, 256, 2, True]           
  7                  -1  1   1180672  ultralytics.nn.modules.conv.Conv             [256

## Измерим параметры на ГПУ

In [3]:
# Validate the model
# При валлидации происходит Fusing слоев модели!
# было 261 слой, стало 195 слоев ! 
# уменьшение слоев примерно на 25%
metrics = model.val(data="ms_coco_val_2017.yaml", device=0)

Ultralytics YOLOv8.0.200 🚀 Python-3.10.13 torch-2.1.0+cu121 CUDA:0 (NVIDIA A10, 24074MiB)
YOLOv8s-seg summary (fused): 195 layers, 11810560 parameters, 0 gradients, 42.6 GFLOPs
val: Scanning /home/admin-gpu/Downloads/yolo_VIKA/homework5/segmentation/datasets/ms_coco_val_2017/val/labels.cache... 4952 images, 48 backgrounds, 0 corrupt: 100%|██████████| 5000/5000 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 313/313 [00:26<00:00, 12.00it/s]
                   all       5000      36335      0.692      0.555      0.609      0.445      0.683      0.536       0.58      0.371
                person       5000      10777      0.801      0.712      0.797      0.575      0.796      0.694      0.774      0.465
               bicycle       5000        314       0.75      0.481      0.574      0.337      0.666      0.411      0.462      0.194
                   car       5000    

## Измерим параметры на ЦПУ

In [4]:
metrics = model.val(data="ms_coco_val_2017.yaml", device='cpu')

Ultralytics YOLOv8.0.200 🚀 Python-3.10.13 torch-2.1.0+cu121 CPU (12th Gen Intel Core(TM) i5-12600)
val: Scanning /home/admin-gpu/Downloads/yolo_VIKA/homework5/segmentation/datasets/ms_coco_val_2017/val/labels.cache... 4952 images, 48 backgrounds, 0 corrupt: 100%|██████████| 5000/5000 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 313/313 [07:11<00:00,  1.38s/it]
                   all       5000      36335      0.692      0.555      0.609      0.445      0.684      0.536       0.58      0.371
                person       5000      10777      0.801      0.712      0.797      0.575      0.798      0.694      0.774      0.465
               bicycle       5000        314      0.751      0.481      0.573      0.337      0.666      0.411      0.462      0.194
                   car       5000       1918      0.717      0.588      0.652      0.443      0.704      0.564     

In [5]:
# оценим метрики для сегментации
print(f"mask map50-95 {metrics.seg.map:.3}")
print(f"mask map50 {metrics.seg.map50:.3}")
print(f"mask map75 {metrics.seg.map75:.3}")

mask map50-95 0.371
mask map50 0.58
mask map75 0.396


In [6]:
# оценим размер сериализованной модели
model_size = os.path.getsize("yolov8s-seg.pt") / 1024**2
print(f"{model_size:.3} Мб")

22.8 Мб


# Экспорт моделей в различные форматы

Согласно официальной документации и туториалу `YOLOv8` поддерживает экспорт в множество различных форматов. 

Export a YOLOv8 model to any supported format below with the `format` argument, i.e. `format=onnx`. See [YOLOv8 Export Docs](https://docs.ultralytics.com/modes/export/) for more information.

- 💡 ProTip: Export to [ONNX](https://onnx.ai/) or [OpenVINO](https://docs.openvino.ai/latest/index.html) for up to 3x CPU speedup.  
- 💡 ProTip: Export to [TensorRT](https://developer.nvidia.com/tensorrt) for up to 5x GPU speedup.


| Format                                                             | `format` Argument | Model                     | Metadata | Arguments                                           |
|--------------------------------------------------------------------|-------------------|---------------------------|----------|-----------------------------------------------------|
| [PyTorch](https://pytorch.org/)                                    | -                 | `yolov8n.pt`              | ✅        | -                                                   |
| [TorchScript](https://pytorch.org/docs/stable/jit.html)            | `torchscript`     | `yolov8n.torchscript`     | ✅        | `imgsz`, `optimize`                                 |
| [ONNX](https://onnx.ai/)                                           | `onnx`            | `yolov8n.onnx`            | ✅        | `imgsz`, `half`, `dynamic`, `simplify`, `opset`     |
| [OpenVINO](https://docs.openvino.ai/latest/index.html)             | `openvino`        | `yolov8n_openvino_model/` | ✅        | `imgsz`, `half`                                     |
| [TensorRT](https://developer.nvidia.com/tensorrt)                  | `engine`          | `yolov8n.engine`          | ✅        | `imgsz`, `half`, `dynamic`, `simplify`, `workspace` |
| [CoreML](https://github.com/apple/coremltools)                     | `coreml`          | `yolov8n.mlpackage`       | ✅        | `imgsz`, `half`, `int8`, `nms`                      |
| [TF SavedModel](https://www.tensorflow.org/guide/saved_model)      | `saved_model`     | `yolov8n_saved_model/`    | ✅        | `imgsz`, `keras`                                    |
| [TF GraphDef](https://www.tensorflow.org/api_docs/python/tf/Graph) | `pb`              | `yolov8n.pb`              | ❌        | `imgsz`                                             |
| [TF Lite](https://www.tensorflow.org/lite)                         | `tflite`          | `yolov8n.tflite`          | ✅        | `imgsz`, `half`, `int8`                             |
| [TF Edge TPU](https://coral.ai/docs/edgetpu/models-intro/)         | `edgetpu`         | `yolov8n_edgetpu.tflite`  | ✅        | `imgsz`                                             |
| [TF.js](https://www.tensorflow.org/js)                             | `tfjs`            | `yolov8n_web_model/`      | ✅        | `imgsz`                                             |
| [PaddlePaddle](https://github.com/PaddlePaddle)                    | `paddle`          | `yolov8n_paddle_model/`   | ✅        | `imgsz`                                             |
| [ncnn](https://github.com/Tencent/ncnn)                            | `ncnn`            | `yolov8n_ncnn_model/`     | ✅        | `imgsz`, `half`                                     |


# Экспорт в ONNX

## ONNX only

In [7]:
# создадим чистую исходную модель

from ultralytics import YOLO
import os

# Load a model
# build a new model from scratch
model = YOLO("yolov8s-seg.yaml")
# load a pretrained model (recommended for training)
model = YOLO("yolov8s-seg.pt")


                   from  n    params  module                                       arguments                     
  0                  -1  1       928  ultralytics.nn.modules.conv.Conv             [3, 32, 3, 2]                 
  1                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  2                  -1  1     29056  ultralytics.nn.modules.block.C2f             [64, 64, 1, True]             
  3                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  4                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  5                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128, 256, 3, 2]              
  6                  -1  2    788480  ultralytics.nn.modules.block.C2f             [256, 256, 2, True]           
  7                  -1  1   1180672  ultralytics.nn.modules.conv.Conv             [256

In [8]:
model.export(format="onnx")

Ultralytics YOLOv8.0.200 🚀 Python-3.10.13 torch-2.1.0+cu121 CPU (12th Gen Intel Core(TM) i5-12600)
YOLOv8s-seg summary (fused): 195 layers, 11810560 parameters, 0 gradients, 42.6 GFLOPs

PyTorch: starting from 'yolov8s-seg.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) ((1, 116, 8400), (1, 32, 160, 160)) (22.8 MB)

ONNX: starting export with onnx 1.14.1 opset 17...
ONNX: export success ✅ 0.5s, saved as 'yolov8s-seg.onnx' (45.3 MB)

Export complete (1.8s)
Results saved to /home/admin-gpu/Downloads/yolo_VIKA/homework5/segmentation
Predict:         yolo predict task=segment model=yolov8s-seg.onnx imgsz=640  
Validate:        yolo val task=segment model=yolov8s-seg.onnx imgsz=640 data=coco.yaml  
Visualize:       https://netron.app


'yolov8s-seg.onnx'

In [9]:
# Load a model
model = YOLO("yolov8s-seg.onnx")

## Измерим параметры на ГПУ

In [10]:
metrics = model.val(data="ms_coco_val_2017.yaml", imgsz=640, device=0)

Ultralytics YOLOv8.0.200 🚀 Python-3.10.13 torch-2.1.0+cu121 CUDA:0 (NVIDIA A10, 24074MiB)
Loading yolov8s-seg.onnx for ONNX Runtime inference...
Forcing batch=1 square inference (1,3,640,640) for non-PyTorch models
val: Scanning /home/admin-gpu/Downloads/yolo_VIKA/homework5/segmentation/datasets/ms_coco_val_2017/val/labels.cache... 4952 images, 48 backgrounds, 0 corrupt: 100%|██████████| 5000/5000 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 5000/5000 [08:49<00:00,  9.44it/s]
                   all       5000      36335      0.692      0.552      0.606      0.444      0.686      0.531      0.577       0.37
                person       5000      10777      0.798      0.711      0.796      0.574      0.797      0.691      0.772      0.464
               bicycle       5000        314      0.728      0.465      0.567      0.335      0.644      0.398      0.462       0.

### Измерим параметры на ЦПУ

In [11]:
metrics = model.val(data="ms_coco_val_2017.yaml", imgsz=640, device='cpu')

Ultralytics YOLOv8.0.200 🚀 Python-3.10.13 torch-2.1.0+cu121 CPU (12th Gen Intel Core(TM) i5-12600)
Loading yolov8s-seg.onnx for ONNX Runtime inference...
Forcing batch=1 square inference (1,3,640,640) for non-PyTorch models
val: Scanning /home/admin-gpu/Downloads/yolo_VIKA/homework5/segmentation/datasets/ms_coco_val_2017/val/labels.cache... 4952 images, 48 backgrounds, 0 corrupt: 100%|██████████| 5000/5000 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 5000/5000 [08:58<00:00,  9.28it/s]
                   all       5000      36335      0.692      0.552      0.606      0.444      0.686      0.531      0.577       0.37
                person       5000      10777      0.798      0.711      0.796      0.574      0.797      0.691      0.772      0.464
               bicycle       5000        314      0.728      0.465      0.567      0.335      0.644      0.398      0.462

In [12]:
# оценим метрики для сегментации
print(f"mask map50-95 {metrics.seg.map:.3}")
print(f"mask map50 {metrics.seg.map50:.3}")
print(f"mask map75 {metrics.seg.map75:.3}")

mask map50-95 0.37
mask map50 0.577
mask map75 0.395


In [13]:
# оценим размер сериализованной модели
model_size = os.path.getsize("yolov8s-seg.onnx") / 1024**2
print(f"{model_size:.3} Мб")

45.3 Мб


In [14]:
os.remove("yolov8s-seg.onnx")

## ONNX simplify

In [15]:
# создадим чистую исходную модель

from ultralytics import YOLO
import os

# Load a model
# build a new model from scratch
model = YOLO("yolov8s-seg.yaml")
# load a pretrained model (recommended for training)
model = YOLO("yolov8s-seg.pt")


                   from  n    params  module                                       arguments                     
  0                  -1  1       928  ultralytics.nn.modules.conv.Conv             [3, 32, 3, 2]                 
  1                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  2                  -1  1     29056  ultralytics.nn.modules.block.C2f             [64, 64, 1, True]             
  3                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  4                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  5                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128, 256, 3, 2]              
  6                  -1  2    788480  ultralytics.nn.modules.block.C2f             [256, 256, 2, True]           
  7                  -1  1   1180672  ultralytics.nn.modules.conv.Conv             [256

In [16]:
model.export(format="onnx", simplify=True)

Ultralytics YOLOv8.0.200 🚀 Python-3.10.13 torch-2.1.0+cu121 CPU (12th Gen Intel Core(TM) i5-12600)
YOLOv8s-seg summary (fused): 195 layers, 11810560 parameters, 0 gradients, 42.6 GFLOPs

PyTorch: starting from 'yolov8s-seg.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) ((1, 116, 8400), (1, 32, 160, 160)) (22.8 MB)

ONNX: starting export with onnx 1.14.1 opset 17...
ONNX: simplifying with onnxsim 0.4.35...
ONNX: export success ✅ 0.9s, saved as 'yolov8s-seg.onnx' (45.2 MB)

Export complete (2.2s)
Results saved to /home/admin-gpu/Downloads/yolo_VIKA/homework5/segmentation
Predict:         yolo predict task=segment model=yolov8s-seg.onnx imgsz=640  
Validate:        yolo val task=segment model=yolov8s-seg.onnx imgsz=640 data=coco.yaml  
Visualize:       https://netron.app


'yolov8s-seg.onnx'

In [17]:
# Load a model
model = YOLO("yolov8s-seg.onnx")

### Измерим параметры на ЦПУ

In [18]:
metrics = model.val(data="ms_coco_val_2017.yaml", imgsz=640, device='cpu')

Ultralytics YOLOv8.0.200 🚀 Python-3.10.13 torch-2.1.0+cu121 CPU (12th Gen Intel Core(TM) i5-12600)
Loading yolov8s-seg.onnx for ONNX Runtime inference...
Forcing batch=1 square inference (1,3,640,640) for non-PyTorch models
val: Scanning /home/admin-gpu/Downloads/yolo_VIKA/homework5/segmentation/datasets/ms_coco_val_2017/val/labels.cache... 4952 images, 48 backgrounds, 0 corrupt: 100%|██████████| 5000/5000 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 5000/5000 [08:33<00:00,  9.73it/s]
                   all       5000      36335      0.692      0.552      0.606      0.444      0.686      0.531      0.577       0.37
                person       5000      10777      0.798      0.711      0.796      0.574      0.797      0.691      0.772      0.464
               bicycle       5000        314      0.728      0.465      0.567      0.335      0.644      0.398      0.462

In [19]:
# оценим метрики для сегментации
print(f"mask map50-95 {metrics.seg.map:.3}")
print(f"mask map50 {metrics.seg.map50:.3}")
print(f"mask map75 {metrics.seg.map75:.3}")

mask map50-95 0.37
mask map50 0.577
mask map75 0.395


In [20]:
# оценим размер сериализованной модели
model_size = os.path.getsize("yolov8s-seg.onnx") / 1024**2
print(f"{model_size:.3} Мб")

45.2 Мб


In [21]:
os.remove("yolov8s-seg.onnx")

## ONNX simplify half

In [22]:
# создадим чистую исходную модель

from ultralytics import YOLO
import os

# Load a model
# build a new model from scratch
model = YOLO("yolov8s-seg.yaml")
# load a pretrained model (recommended for training)
model = YOLO("yolov8s-seg.pt")


                   from  n    params  module                                       arguments                     
  0                  -1  1       928  ultralytics.nn.modules.conv.Conv             [3, 32, 3, 2]                 
  1                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  2                  -1  1     29056  ultralytics.nn.modules.block.C2f             [64, 64, 1, True]             
  3                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  4                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  5                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128, 256, 3, 2]              
  6                  -1  2    788480  ultralytics.nn.modules.block.C2f             [256, 256, 2, True]           
  7                  -1  1   1180672  ultralytics.nn.modules.conv.Conv             [256

In [23]:
model.export(format="onnx", simplify=True, half=True)

Ultralytics YOLOv8.0.200 🚀 Python-3.10.13 torch-2.1.0+cu121 CPU (12th Gen Intel Core(TM) i5-12600)
WARNING ⚠️ half=True only compatible with GPU export, i.e. use device=0
YOLOv8s-seg summary (fused): 195 layers, 11810560 parameters, 0 gradients, 42.6 GFLOPs

PyTorch: starting from 'yolov8s-seg.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) ((1, 116, 8400), (1, 32, 160, 160)) (22.8 MB)

ONNX: starting export with onnx 1.14.1 opset 17...
ONNX: simplifying with onnxsim 0.4.35...
ONNX: export success ✅ 0.8s, saved as 'yolov8s-seg.onnx' (45.2 MB)

Export complete (2.2s)
Results saved to /home/admin-gpu/Downloads/yolo_VIKA/homework5/segmentation
Predict:         yolo predict task=segment model=yolov8s-seg.onnx imgsz=640  
Validate:        yolo val task=segment model=yolov8s-seg.onnx imgsz=640 data=coco.yaml  
Visualize:       https://netron.app


'yolov8s-seg.onnx'

In [24]:
# Load a model
model = YOLO("yolov8s-seg.onnx")

### Измерим параметры на ЦПУ

In [25]:
metrics = model.val(data="ms_coco_val_2017.yaml", imgsz=640, device='cpu')

Ultralytics YOLOv8.0.200 🚀 Python-3.10.13 torch-2.1.0+cu121 CPU (12th Gen Intel Core(TM) i5-12600)
Loading yolov8s-seg.onnx for ONNX Runtime inference...
Forcing batch=1 square inference (1,3,640,640) for non-PyTorch models
val: Scanning /home/admin-gpu/Downloads/yolo_VIKA/homework5/segmentation/datasets/ms_coco_val_2017/val/labels.cache... 4952 images, 48 backgrounds, 0 corrupt: 100%|██████████| 5000/5000 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 5000/5000 [08:47<00:00,  9.48it/s]
                   all       5000      36335      0.692      0.552      0.606      0.444      0.686      0.531      0.577       0.37
                person       5000      10777      0.798      0.711      0.796      0.574      0.797      0.691      0.772      0.464
               bicycle       5000        314      0.728      0.465      0.567      0.335      0.644      0.398      0.462

In [26]:
# оценим метрики для сегментации
print(f"mask map50-95 {metrics.seg.map:.3}")
print(f"mask map50 {metrics.seg.map50:.3}")
print(f"mask map75 {metrics.seg.map75:.3}")

mask map50-95 0.37
mask map50 0.577
mask map75 0.395


In [27]:
# оценим размер сериализованной модели
model_size = os.path.getsize("yolov8s-seg.onnx") / 1024**2
print(f"{model_size:.3} Мб")

45.2 Мб


In [28]:
os.remove("yolov8s-seg.onnx")

# Экспорт в TensorRT

## TensorRT

In [57]:
# создадим чистую исходную модель

from ultralytics import YOLO
import os

# Load a model
# build a new model from scratch
model = YOLO("yolov8s-seg.yaml")
# load a pretrained model (recommended for training)
model = YOLO("yolov8s-seg.pt")


                   from  n    params  module                                       arguments                     
  0                  -1  1       928  ultralytics.nn.modules.conv.Conv             [3, 32, 3, 2]                 
  1                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  2                  -1  1     29056  ultralytics.nn.modules.block.C2f             [64, 64, 1, True]             
  3                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  4                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  5                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128, 256, 3, 2]              
  6                  -1  2    788480  ultralytics.nn.modules.block.C2f             [256, 256, 2, True]           
  7                  -1  1   1180672  ultralytics.nn.modules.conv.Conv             [256

In [58]:
model.export(format="engine", simplify=False)

WARNING ⚠️ TensorRT requires GPU export, automatically assigning device=0
Ultralytics YOLOv8.0.200 🚀 Python-3.10.13 torch-2.1.0+cu121 CUDA:0 (NVIDIA A10, 24074MiB)
YOLOv8s-seg summary (fused): 195 layers, 11810560 parameters, 0 gradients, 42.6 GFLOPs

PyTorch: starting from 'yolov8s-seg.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) ((1, 116, 8400), (1, 32, 160, 160)) (22.8 MB)

ONNX: starting export with onnx 1.14.1 opset 17...
ONNX: simplifying with onnxsim 0.4.35...
ONNX: export success ✅ 0.8s, saved as 'yolov8s-seg.onnx' (45.2 MB)

TensorRT: starting export with TensorRT 8.6.1...
TensorRT: input "images" with shape(1, 3, 640, 640) DataType.FLOAT
TensorRT: output "output0" with shape(1, 116, 8400) DataType.FLOAT
TensorRT: output "output1" with shape(1, 32, 160, 160) DataType.FLOAT
TensorRT: building FP32 engine as yolov8s-seg.engine


[10/23/2023-05:49:13] [TRT] [I] The logger passed into createInferBuilder differs from one already provided for an existing builder, runtime, or refitter. Uses of the global logger, returned by nvinfer1::getLogger(), will return the existing value.
[10/23/2023-05:49:13] [TRT] [I] [MemUsageChange] Init CUDA: CPU +0, GPU +0, now: CPU 5360, GPU 9542 (MiB)
[10/23/2023-05:49:16] [TRT] [I] [MemUsageChange] Init builder kernel library: CPU +1214, GPU +266, now: CPU 6574, GPU 9808 (MiB)
[10/23/2023-05:49:16] [TRT] [W] CUDA lazy loading is not enabled. Enabling it can significantly reduce device memory usage and speed up TensorRT initialization. See "Lazy Loading" section of CUDA documentation https://docs.nvidia.com/cuda/cuda-c-programming-guide/index.html#lazy-loading
[10/23/2023-05:49:16] [TRT] [I] ----------------------------------------------------------------
[10/23/2023-05:49:16] [TRT] [I] Input filename:   yolov8s-seg.onnx
[10/23/2023-05:49:16] [TRT] [I] ONNX IR version:  0.0.8
[10/23/2

TensorRT: export success ✅ 62.2s, saved as 'yolov8s-seg.engine' (55.0 MB)

Export complete (62.3s)
Results saved to /home/admin-gpu/Downloads/yolo_VIKA/homework5/segmentation
Predict:         yolo predict task=segment model=yolov8s-seg.engine imgsz=640  
Validate:        yolo val task=segment model=yolov8s-seg.engine imgsz=640 data=coco.yaml  
Visualize:       https://netron.app


'yolov8s-seg.engine'

In [60]:
# Load a model
model = YOLO("yolov8s-seg.engine")

### Измерим параметры на ГПУ

In [61]:
metrics = model.val(data="ms_coco_val_2017.yaml", imgsz=640, device=0)

Ultralytics YOLOv8.0.200 🚀 Python-3.10.13 torch-2.1.0+cu121 CUDA:0 (NVIDIA A10, 24074MiB)
Loading yolov8s-seg.engine for TensorRT inference...


[10/23/2023-05:50:25] [TRT] [I] The logger passed into createInferRuntime differs from one already provided for an existing builder, runtime, or refitter. Uses of the global logger, returned by nvinfer1::getLogger(), will return the existing value.
[10/23/2023-05:50:25] [TRT] [I] Loaded engine size: 54 MiB
[10/23/2023-05:50:25] [TRT] [I] [MemUsageChange] TensorRT-managed allocation in engine deserialization: CPU +0, GPU +52, now: CPU 0, GPU 142 (MiB)
[10/23/2023-05:50:25] [TRT] [I] [MemUsageChange] TensorRT-managed allocation in IExecutionContext creation: CPU +1, GPU +39, now: CPU 1, GPU 181 (MiB)
[10/23/2023-05:50:25] [TRT] [W] CUDA lazy loading is not enabled. Enabling it can significantly reduce device memory usage and speed up TensorRT initialization. See "Lazy Loading" section of CUDA documentation https://docs.nvidia.com/cuda/cuda-c-programming-guide/index.html#lazy-loading


val: Scanning /home/admin-gpu/Downloads/yolo_VIKA/homework5/segmentation/datasets/ms_coco_val_2017/val/labels.cache... 4952 images, 48 backgrounds, 0 corrupt: 100%|██████████| 5000/5000 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 5000/5000 [00:58<00:00, 85.47it/s]
                   all       5000      36335      0.693      0.552      0.606      0.444      0.687      0.531      0.577       0.37
                person       5000      10777      0.798      0.711      0.796      0.574      0.798       0.69      0.772      0.464
               bicycle       5000        314       0.73      0.465      0.566      0.335      0.645      0.398      0.462      0.191
                   car       5000       1918      0.726      0.584      0.652       0.44      0.713      0.555      0.615       0.35
            motorcycle       5000        367      0.784      0.649      0.745  

In [62]:
# оценим метрики для сегментации
print(f"mask map50-95 {metrics.seg.map:.3}")
print(f"mask map50 {metrics.seg.map50:.3}")
print(f"mask map75 {metrics.seg.map75:.3}")

mask map50-95 0.37
mask map50 0.577
mask map75 0.395


In [64]:
# оценим размер сериализованной модели
model_size = os.path.getsize("yolov8s-seg.engine") / 1024**2
print(f"{model_size:.3} Мб")

55.0 Мб


In [65]:
os.remove("yolov8s-seg.engine")

## TensorRT Simplify

In [50]:
# создадим чистую исходную модель

from ultralytics import YOLO
import os

# Load a model
# build a new model from scratch
model = YOLO("yolov8s-seg.yaml")
# load a pretrained model (recommended for training)
model = YOLO("yolov8s-seg.pt")


                   from  n    params  module                                       arguments                     
  0                  -1  1       928  ultralytics.nn.modules.conv.Conv             [3, 32, 3, 2]                 
  1                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  2                  -1  1     29056  ultralytics.nn.modules.block.C2f             [64, 64, 1, True]             
  3                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  4                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  5                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128, 256, 3, 2]              
  6                  -1  2    788480  ultralytics.nn.modules.block.C2f             [256, 256, 2, True]           
  7                  -1  1   1180672  ultralytics.nn.modules.conv.Conv             [256

In [51]:
model.export(format="engine", simplify=True)

WARNING ⚠️ TensorRT requires GPU export, automatically assigning device=0
Ultralytics YOLOv8.0.200 🚀 Python-3.10.13 torch-2.1.0+cu121 CUDA:0 (NVIDIA A10, 24074MiB)
YOLOv8s-seg summary (fused): 195 layers, 11810560 parameters, 0 gradients, 42.6 GFLOPs

PyTorch: starting from 'yolov8s-seg.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) ((1, 116, 8400), (1, 32, 160, 160)) (22.8 MB)

ONNX: starting export with onnx 1.14.1 opset 17...
ONNX: simplifying with onnxsim 0.4.35...
ONNX: export success ✅ 0.8s, saved as 'yolov8s-seg.onnx' (45.2 MB)

TensorRT: starting export with TensorRT 8.6.1...
TensorRT: input "images" with shape(1, 3, 640, 640) DataType.FLOAT
TensorRT: output "output0" with shape(1, 116, 8400) DataType.FLOAT
TensorRT: output "output1" with shape(1, 32, 160, 160) DataType.FLOAT
TensorRT: building FP32 engine as yolov8s-seg.engine


[10/23/2023-05:45:26] [TRT] [I] [MemUsageChange] Init CUDA: CPU +0, GPU +0, now: CPU 5123, GPU 9350 (MiB)
[10/23/2023-05:45:29] [TRT] [I] [MemUsageChange] Init builder kernel library: CPU +1216, GPU +268, now: CPU 6339, GPU 9618 (MiB)
[10/23/2023-05:45:29] [TRT] [W] CUDA lazy loading is not enabled. Enabling it can significantly reduce device memory usage and speed up TensorRT initialization. See "Lazy Loading" section of CUDA documentation https://docs.nvidia.com/cuda/cuda-c-programming-guide/index.html#lazy-loading
[10/23/2023-05:45:29] [TRT] [I] ----------------------------------------------------------------
[10/23/2023-05:45:29] [TRT] [I] Input filename:   yolov8s-seg.onnx
[10/23/2023-05:45:29] [TRT] [I] ONNX IR version:  0.0.8
[10/23/2023-05:45:29] [TRT] [I] Opset version:    17
[10/23/2023-05:45:29] [TRT] [I] Producer name:    pytorch
[10/23/2023-05:45:29] [TRT] [I] Producer version: 2.1.0
[10/23/2023-05:45:29] [TRT] [I] Domain:           
[10/23/2023-05:45:29] [TRT] [I] Model v

TensorRT: export success ✅ 86.8s, saved as 'yolov8s-seg.engine' (54.8 MB)

Export complete (87.0s)
Results saved to /home/admin-gpu/Downloads/yolo_VIKA/homework5/segmentation
Predict:         yolo predict task=segment model=yolov8s-seg.engine imgsz=640  
Validate:        yolo val task=segment model=yolov8s-seg.engine imgsz=640 data=coco.yaml  
Visualize:       https://netron.app


'yolov8s-seg.engine'

In [52]:
# Load a model
model = YOLO("yolov8s-seg.engine")

### Измерим параметры на ГПУ

In [53]:
metrics = model.val(data="ms_coco_val_2017.yaml", imgsz=640, device=0)

Ultralytics YOLOv8.0.200 🚀 Python-3.10.13 torch-2.1.0+cu121 CUDA:0 (NVIDIA A10, 24074MiB)
Loading yolov8s-seg.engine for TensorRT inference...


[10/23/2023-05:47:34] [TRT] [I] Loaded engine size: 54 MiB
[10/23/2023-05:47:34] [TRT] [I] [MemUsageChange] TensorRT-managed allocation in engine deserialization: CPU +0, GPU +51, now: CPU 0, GPU 51 (MiB)
[10/23/2023-05:47:34] [TRT] [I] [MemUsageChange] TensorRT-managed allocation in IExecutionContext creation: CPU +0, GPU +39, now: CPU 0, GPU 90 (MiB)
[10/23/2023-05:47:34] [TRT] [W] CUDA lazy loading is not enabled. Enabling it can significantly reduce device memory usage and speed up TensorRT initialization. See "Lazy Loading" section of CUDA documentation https://docs.nvidia.com/cuda/cuda-c-programming-guide/index.html#lazy-loading


val: Scanning /home/admin-gpu/Downloads/yolo_VIKA/homework5/segmentation/datasets/ms_coco_val_2017/val/labels.cache... 4952 images, 48 backgrounds, 0 corrupt: 100%|██████████| 5000/5000 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 5000/5000 [00:57<00:00, 86.57it/s]
                   all       5000      36335      0.693      0.552      0.606      0.444      0.687      0.531      0.577       0.37
                person       5000      10777      0.798      0.711      0.796      0.574      0.798       0.69      0.772      0.464
               bicycle       5000        314       0.73      0.465      0.566      0.335      0.645      0.398      0.462      0.191
                   car       5000       1918      0.726      0.584      0.652       0.44      0.713      0.555      0.615       0.35
            motorcycle       5000        367      0.784      0.649      0.745  

In [54]:
# оценим метрики для сегментации
print(f"mask map50-95 {metrics.seg.map:.3}")
print(f"mask map50 {metrics.seg.map50:.3}")
print(f"mask map75 {metrics.seg.map75:.3}")

mask map50-95 0.37
mask map50 0.577
mask map75 0.395


In [55]:
# оценим размер сериализованной модели
model_size = os.path.getsize("yolov8s-seg.engine") / 1024**2
print(f"{model_size:.3} Мб")

54.8 Мб


In [56]:
os.remove("yolov8s-seg.engine")

## TensorRT Simplify Half

In [29]:
# создадим чистую исходную модель

from ultralytics import YOLO
import os

# Load a model
# build a new model from scratch
model = YOLO("yolov8s-seg.yaml")
# load a pretrained model (recommended for training)
model = YOLO("yolov8s-seg.pt")


                   from  n    params  module                                       arguments                     
  0                  -1  1       928  ultralytics.nn.modules.conv.Conv             [3, 32, 3, 2]                 
  1                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  2                  -1  1     29056  ultralytics.nn.modules.block.C2f             [64, 64, 1, True]             
  3                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  4                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  5                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128, 256, 3, 2]              
  6                  -1  2    788480  ultralytics.nn.modules.block.C2f             [256, 256, 2, True]           
  7                  -1  1   1180672  ultralytics.nn.modules.conv.Conv             [256

In [30]:
model.export(format="engine", simplify=True, half=True)

WARNING ⚠️ TensorRT requires GPU export, automatically assigning device=0
Ultralytics YOLOv8.0.200 🚀 Python-3.10.13 torch-2.1.0+cu121 CUDA:0 (NVIDIA A10, 24074MiB)
YOLOv8s-seg summary (fused): 195 layers, 11810560 parameters, 0 gradients, 42.6 GFLOPs

PyTorch: starting from 'yolov8s-seg.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) ((1, 116, 8400), (1, 32, 160, 160)) (22.8 MB)

ONNX: starting export with onnx 1.14.1 opset 17...
ONNX: simplifying with onnxsim 0.4.35...
ONNX: export success ✅ 0.6s, saved as 'yolov8s-seg.onnx' (22.6 MB)

TensorRT: starting export with TensorRT 8.6.1...
TensorRT: input "images" with shape(1, 3, 640, 640) DataType.HALF
TensorRT: output "output0" with shape(1, 116, 8400) DataType.HALF
TensorRT: output "output1" with shape(1, 32, 160, 160) DataType.HALF
TensorRT: building FP16 engine as yolov8s-seg.engine


[10/23/2023-05:24:23] [TRT] [I] [MemUsageChange] Init CUDA: CPU +349, GPU +0, now: CPU 5518, GPU 12328 (MiB)
[10/23/2023-05:24:26] [TRT] [I] [MemUsageChange] Init builder kernel library: CPU +1218, GPU +268, now: CPU 6812, GPU 12596 (MiB)
[10/23/2023-05:24:26] [TRT] [W] CUDA lazy loading is not enabled. Enabling it can significantly reduce device memory usage and speed up TensorRT initialization. See "Lazy Loading" section of CUDA documentation https://docs.nvidia.com/cuda/cuda-c-programming-guide/index.html#lazy-loading
[10/23/2023-05:24:26] [TRT] [I] ----------------------------------------------------------------
[10/23/2023-05:24:26] [TRT] [I] Input filename:   yolov8s-seg.onnx
[10/23/2023-05:24:26] [TRT] [I] ONNX IR version:  0.0.8
[10/23/2023-05:24:26] [TRT] [I] Opset version:    17
[10/23/2023-05:24:26] [TRT] [I] Producer name:    pytorch
[10/23/2023-05:24:26] [TRT] [I] Producer version: 2.1.0
[10/23/2023-05:24:26] [TRT] [I] Domain:           
[10/23/2023-05:24:26] [TRT] [I] Mod

TensorRT: export success ✅ 248.1s, saved as 'yolov8s-seg.engine' (25.9 MB)

Export complete (248.2s)
Results saved to /home/admin-gpu/Downloads/yolo_VIKA/homework5/segmentation
Predict:         yolo predict task=segment model=yolov8s-seg.engine imgsz=640 half 
Validate:        yolo val task=segment model=yolov8s-seg.engine imgsz=640 data=coco.yaml half 
Visualize:       https://netron.app


'yolov8s-seg.engine'

In [31]:
# Load a model
model = YOLO("yolov8s-seg.engine")

### Измерим параметры на ГПУ

In [32]:
metrics = model.val(data="ms_coco_val_2017.yaml", imgsz=640, device=0)

Ultralytics YOLOv8.0.200 🚀 Python-3.10.13 torch-2.1.0+cu121 CUDA:0 (NVIDIA A10, 24074MiB)
Loading yolov8s-seg.engine for TensorRT inference...


[10/23/2023-05:28:30] [TRT] [I] Loaded engine size: 25 MiB
[10/23/2023-05:28:30] [TRT] [I] [MemUsageChange] TensorRT-managed allocation in engine deserialization: CPU +0, GPU +22, now: CPU 0, GPU 22 (MiB)
[10/23/2023-05:28:30] [TRT] [I] [MemUsageChange] TensorRT-managed allocation in IExecutionContext creation: CPU +0, GPU +21, now: CPU 0, GPU 43 (MiB)
[10/23/2023-05:28:30] [TRT] [W] CUDA lazy loading is not enabled. Enabling it can significantly reduce device memory usage and speed up TensorRT initialization. See "Lazy Loading" section of CUDA documentation https://docs.nvidia.com/cuda/cuda-c-programming-guide/index.html#lazy-loading


val: Scanning /home/admin-gpu/Downloads/yolo_VIKA/homework5/segmentation/datasets/ms_coco_val_2017/val/labels.cache... 4952 images, 48 backgrounds, 0 corrupt: 100%|██████████| 5000/5000 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 5000/5000 [00:48<00:00, 103.93it/s]
                   all       5000      36335      0.693      0.552      0.606      0.443       0.69      0.532      0.579      0.371
                person       5000      10777      0.797      0.711      0.795      0.573        0.8      0.692      0.775      0.467
               bicycle       5000        314      0.734      0.468      0.566      0.337      0.652      0.398      0.458       0.19
                   car       5000       1918      0.728      0.585      0.651      0.441      0.718      0.554      0.617      0.354
            motorcycle       5000        367      0.781      0.649      0.744 

In [33]:
# оценим метрики для сегментации
print(f"mask map50-95 {metrics.seg.map:.3}")
print(f"mask map50 {metrics.seg.map50:.3}")
print(f"mask map75 {metrics.seg.map75:.3}")

mask map50-95 0.371
mask map50 0.579
mask map75 0.397


In [34]:
# оценим размер сериализованной модели
model_size = os.path.getsize("yolov8s-seg.engine") / 1024**2
print(f"{model_size:.3} Мб")

25.9 Мб


In [35]:
os.remove("yolov8s-seg.engine")

# Экспорт в OpenVino

## OpenVino

In [36]:
# создадим чистую исходную модель

from ultralytics import YOLO
import os

# Load a model
# build a new model from scratch
model = YOLO("yolov8s-seg.yaml")
# load a pretrained model (recommended for training)
model = YOLO("yolov8s-seg.pt")


                   from  n    params  module                                       arguments                     
  0                  -1  1       928  ultralytics.nn.modules.conv.Conv             [3, 32, 3, 2]                 
  1                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  2                  -1  1     29056  ultralytics.nn.modules.block.C2f             [64, 64, 1, True]             
  3                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  4                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  5                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128, 256, 3, 2]              
  6                  -1  2    788480  ultralytics.nn.modules.block.C2f             [256, 256, 2, True]           
  7                  -1  1   1180672  ultralytics.nn.modules.conv.Conv             [256

In [37]:
model.export(format="openvino")

Ultralytics YOLOv8.0.200 🚀 Python-3.10.13 torch-2.1.0+cu121 CPU (12th Gen Intel Core(TM) i5-12600)
YOLOv8s-seg summary (fused): 195 layers, 11810560 parameters, 0 gradients, 42.6 GFLOPs

PyTorch: starting from 'yolov8s-seg.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) ((1, 116, 8400), (1, 32, 160, 160)) (22.8 MB)

ONNX: starting export with onnx 1.14.1 opset 17...
ONNX: export success ✅ 0.5s, saved as 'yolov8s-seg.onnx' (45.3 MB)

OpenVINO: starting export with openvino 2023.1.0-12185-9e6b00e51cd-releases/2023/1...
OpenVINO: export success ✅ 0.4s, saved as 'yolov8s-seg_openvino_model/' (45.4 MB)

Export complete (2.3s)
Results saved to /home/admin-gpu/Downloads/yolo_VIKA/homework5/segmentation
Predict:         yolo predict task=segment model=yolov8s-seg_openvino_model imgsz=640  
Validate:        yolo val task=segment model=yolov8s-seg_openvino_model imgsz=640 data=coco.yaml  
Visualize:       https://netron.app


'yolov8s-seg_openvino_model'

In [38]:
# Load a model
model = YOLO("yolov8s-seg_openvino_model")

### Измерим параметры на ЦПУ

In [39]:
metrics = model.val(data="ms_coco_val_2017.yaml", imgsz=640, device='cpu')

Ultralytics YOLOv8.0.200 🚀 Python-3.10.13 torch-2.1.0+cu121 CPU (12th Gen Intel Core(TM) i5-12600)
Loading yolov8s-seg_openvino_model for OpenVINO inference...
Forcing batch=1 square inference (1,3,640,640) for non-PyTorch models
val: Scanning /home/admin-gpu/Downloads/yolo_VIKA/homework5/segmentation/datasets/ms_coco_val_2017/val/labels.cache... 4952 images, 48 backgrounds, 0 corrupt: 100%|██████████| 5000/5000 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 5000/5000 [06:25<00:00, 12.96it/s]
                   all       5000      36335      0.692      0.552      0.606      0.444      0.686      0.531      0.577       0.37
                person       5000      10777      0.798      0.711      0.796      0.574      0.797      0.691      0.772      0.464
               bicycle       5000        314      0.728      0.465      0.567      0.335      0.644      0.398     

In [40]:
# оценим метрики для сегментации
print(f"mask map50-95 {metrics.seg.map:.3}")
print(f"mask map50 {metrics.seg.map50:.3}")
print(f"mask map75 {metrics.seg.map75:.3}")

mask map50-95 0.37
mask map50 0.577
mask map75 0.395


In [41]:
# оценим размер сериализованной модели
model_size = sum(os.path.getsize(f) for f in os.listdir('.') if os.path.isfile(f)) / 1024**2
print(f"{model_size:.3} Мб")

68.2 Мб


In [42]:
shutil.rmtree('yolov8s-seg_openvino_model')

## OpenVino half

In [43]:
# создадим чистую исходную модель

from ultralytics import YOLO
import os

# Load a model
# build a new model from scratch
model = YOLO("yolov8s-seg.yaml")
# load a pretrained model (recommended for training)
model = YOLO("yolov8s-seg.pt")


                   from  n    params  module                                       arguments                     
  0                  -1  1       928  ultralytics.nn.modules.conv.Conv             [3, 32, 3, 2]                 
  1                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  2                  -1  1     29056  ultralytics.nn.modules.block.C2f             [64, 64, 1, True]             
  3                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  4                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  5                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128, 256, 3, 2]              
  6                  -1  2    788480  ultralytics.nn.modules.block.C2f             [256, 256, 2, True]           
  7                  -1  1   1180672  ultralytics.nn.modules.conv.Conv             [256

In [44]:
model.export(format="openvino", half=True)

Ultralytics YOLOv8.0.200 🚀 Python-3.10.13 torch-2.1.0+cu121 CPU (12th Gen Intel Core(TM) i5-12600)
YOLOv8s-seg summary (fused): 195 layers, 11810560 parameters, 0 gradients, 42.6 GFLOPs

PyTorch: starting from 'yolov8s-seg.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) ((1, 116, 8400), (1, 32, 160, 160)) (22.8 MB)

ONNX: starting export with onnx 1.14.1 opset 17...
ONNX: export success ✅ 0.7s, saved as 'yolov8s-seg.onnx' (45.3 MB)

OpenVINO: starting export with openvino 2023.1.0-12185-9e6b00e51cd-releases/2023/1...
OpenVINO: export success ✅ 0.1s, saved as 'yolov8s-seg_openvino_model/' (22.9 MB)

Export complete (2.2s)
Results saved to /home/admin-gpu/Downloads/yolo_VIKA/homework5/segmentation
Predict:         yolo predict task=segment model=yolov8s-seg_openvino_model imgsz=640 half 
Validate:        yolo val task=segment model=yolov8s-seg_openvino_model imgsz=640 data=coco.yaml half 
Visualize:       https://netron.app


'yolov8s-seg_openvino_model'

In [45]:
# Load a model
model = YOLO("yolov8s-seg_openvino_model")

### Измерим параметры на ЦПУ

In [46]:
metrics = model.val(data="ms_coco_val_2017.yaml", imgsz=640, device='cpu')

Ultralytics YOLOv8.0.200 🚀 Python-3.10.13 torch-2.1.0+cu121 CPU (12th Gen Intel Core(TM) i5-12600)
Loading yolov8s-seg_openvino_model for OpenVINO inference...
Forcing batch=1 square inference (1,3,640,640) for non-PyTorch models
val: Scanning /home/admin-gpu/Downloads/yolo_VIKA/homework5/segmentation/datasets/ms_coco_val_2017/val/labels.cache... 4952 images, 48 backgrounds, 0 corrupt: 100%|██████████| 5000/5000 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 5000/5000 [06:21<00:00, 13.09it/s]
                   all       5000      36335      0.693      0.552      0.606      0.444      0.688      0.531      0.577       0.37
                person       5000      10777      0.798      0.711      0.796      0.574      0.798       0.69      0.772      0.464
               bicycle       5000        314      0.735      0.468      0.567      0.335      0.647      0.398     

In [47]:
# оценим метрики для сегментации
print(f"mask map50-95 {metrics.seg.map:.3}")
print(f"mask map50 {metrics.seg.map50:.3}")
print(f"mask map75 {metrics.seg.map75:.3}")

mask map50-95 0.37
mask map50 0.577
mask map75 0.395


In [48]:
# оценим размер сериализованной модели
model_size = sum(os.path.getsize(f) for f in os.listdir('.') if os.path.isfile(f)) / 1024**2
print(f"{model_size:.3} Мб")

68.2 Мб


In [49]:
shutil.rmtree('yolov8s-seg_openvino_model')